In [1]:
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
cf = pd.read_csv('./data/cfdata.csv') #전체 카페 데이터
cf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30656 entries, 0 to 30655
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   30656 non-null  int64  
 1   APVPERMYMD   30656 non-null  object 
 2   TRDSTATEGBN  30656 non-null  int64  
 3   TRDSTATENM   30656 non-null  object 
 4   DCBYMD       14603 non-null  object 
 5   SITEAREA     29057 non-null  object 
 6   RDNWHLADDR   30656 non-null  object 
 7   BPLCNM       30656 non-null  object 
 8   UPTAENM      30656 non-null  object 
 9   X            30489 non-null  float64
 10  Y            30489 non-null  float64
 11  region       30656 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 2.8+ MB


In [3]:
date = pd.to_datetime(cf['DCBYMD'],format='%Y-%m-%d')
cf['DCBYMD'] = date
cf_recent = cf[cf['DCBYMD'].dt.year > 2016]  #8353개
cf_open = cf[cf['TRDSTATEGBN'] == 1]        #15142개
cfdata = pd.concat([cf_recent,cf_open])     #23495개
cf_dj = cfdata[cfdata['region']=='동작구']  #705개
#cf_dj.groupby('TRDSTATENM').region.count()
cf_dj

,Unnamed: 0,APVPERMYMD,TRDSTATEGBN,TRDSTATENM,DCBYMD,SITEAREA,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,region
209,209,2014-11-20,3,폐업,2020-12-22,52.80,"서울특별시 동작구 노량진로 206, 1층 (노량진동, 백명트렌디타워)",라떼야커피 노량진점,커피숍,195388.277453,445614.107790,동작구
222,222,2016-10-19,3,폐업,2020-12-22,13.20,"서울특별시 동작구 상도로 95, 지상1층 (상도동)",디베리아카페,커피숍,193823.589569,444223.639902,동작구
223,223,2016-10-12,3,폐업,2020-12-22,12.00,"서울특별시 동작구 사당로 157-2, 지상1층 (사당동)",위드,커피숍,197029.827432,442821.776177,동작구
230,230,2010-05-10,3,폐업,2020-12-22,46.20,"서울특별시 동작구 상도로 370 (상도동,(3층))",프로라이프문화센터&커피(coffee),커피숍,195838.529195,443724.790336,동작구
310,310,2014-08-01,3,폐업,2020-12-22,8.00,"서울특별시 동작구 보라매로19길 5-1, 1층 (신대방동)",테이크아웃 커피전문점 H,커피숍,193516.885414,443917.853118,동작구
...,...,...,...,...,...,...,...,...,...,...,...,...
26136,26136,2012-02-17,1,영업/정상,NaT,16.20,"서울특별시 동작구 노량진로 162 (노량진동, 1층)",카페1987,까페,194972.424068,445686.158478,동작구
26230,26230,2012-05-16,1,영업/정상,NaT,202.00,"서울특별시 동작구 노량진로 144 (노량진동, 외 2필지, 2층)",설빙 노량진점,까페,194787.339913,445690.237396,동작구
26231,26231,2012-05-21,1,영업/정상,NaT,18.00,"서울특별시 동작구 상도로 161 (상도동, 1층)",미모,까페,194346.790618,444619.174597,동작구
26232,26232,2010-10-07,1,영업/정상,NaT,5.17,"서울특별시 동작구 노량진로 162 (노량진동,1층 8-4호)",커피자리,까페,194972.424068,445686.158478,동작구


In [4]:
#폐업 날짜 추가
closed_date = cf_dj['DCBYMD'].reset_index(drop=True)
closed_date

0     2020-12-22
1     2020-12-22
2     2020-12-22
3     2020-12-22
4     2020-12-22
         ...    
702          NaT
703          NaT
704          NaT
705          NaT
706          NaT
Name: DCBYMD, Length: 707, dtype: datetime64[ns]

In [5]:
#좌표정보 추가
x=cf_dj['X'].reset_index(drop=True)
y=cf_dj['Y'].reset_index(drop=True)

x=x.astype('str')
y=y.astype('str')

In [6]:
cf_opcl = cf_dj['TRDSTATENM'].reset_index(drop=True)
cf_opcl

0         폐업
1         폐업
2         폐업
3         폐업
4         폐업
       ...  
702    영업/정상
703    영업/정상
704    영업/정상
705    영업/정상
706    영업/정상
Name: TRDSTATENM, Length: 707, dtype: object

In [7]:
addr = cf_dj.RDNWHLADDR
addr = addr.tolist()
addr[4]

'서울특별시 동작구 보라매로19길 5-1, 1층 (신대방동)'

In [8]:
#url1 = 'https://dapi.kakao.com/v2/local/search/address.json?query='+ addr[11]
#headers = {"Authorization": "KakaoAK 6341d1e76477fbac498a469adc87108d"}
#result1 = json.loads(str(requests.get(url1, headers=headers).text))
#len(result1['documents'])

#i=4(서울특별시 동작구 보라매로19길 5-1, 1층 (신대방동)/주소없음)

In [13]:
url = 'https://dapi.kakao.com/v2/local/geo/transcoord.json?x=197308.865165&y=448745.657721&input_coord=TM&output_coord=WGS84'
headers = {"Authorization":"KakaoAK 6341d1e76477fbac498a469adc87108d"}
result = json.loads(str(requests.get(url, headers=headers).text))
#print(len(result['documents']))
result['documents'][0]

{'x': 126.9703380545782, 'y': 37.54096321541923}

In [50]:
dj_sw8 = pd.DataFrame({'cf_address':[],'X':[],'Y':[],'nb_num':[],'place_name':[],'distance':[],'closed_date':[]})

address=[]
op_cl=[]
cf_x=[]
cf_y=[]
near=[]
cl_date=[]

for i in range(len(x)):
    #2017~폐업한 가게 주소 변환

    url1 = 'https://dapi.kakao.com/v2/local/geo/transcoord.json?x=' + x[i] +'&y='+ y[i] +'&input_coord=TM&output_coord=WGS84'
    headers = {"Authorization": "KakaoAK 6341d1e76477fbac498a469adc87108d"}
    result1 = json.loads(str(requests.get(url1, headers=headers).text))
    x_info = str(result1['documents'][0]['x'])
    y_info = str(result1['documents'][0]['y'])
    
    #카페 주변 1km내 지하철역
    url = 'https://dapi.kakao.com/v2/local/search/category.json?category_group_code=SW8&radius=1000&sort=distance&x='+ x_info + '&y=' + y_info
    result = json.loads(str(requests.get(url, headers=headers).text))

        
    #지하철 정보 및 주소 정보 추가
    dj_sw8_sam = pd.DataFrame([result['documents'][0]],columns=['distance','place_name'])
    address.append(addr[i])
    cf_x.append(result1['documents'][0]['x'])   #카페 데이터의 좌표(WGS84)
    cf_y.append(result1['documents'][0]['y'])
    near.append(len(result['documents']))       #주변 지하철역 개수
    op_cl.append(cf_opcl[i])
    cl_date.append(closed_date[i])
    dj_sw8 = dj_sw8.append(dj_sw8_sam)

        
dj_sw8['cf_address'] = address
dj_sw8['X'] = cf_x
dj_sw8['Y'] = cf_y
dj_sw8['nb_num'] = near
dj_sw8['open_closed'] = op_cl
dj_sw8['closed_date'] = cl_date

In [51]:
dj_sw8.head(10)

,cf_address,X,Y,nb_num,place_name,distance,closed_date,open_closed
0,"서울특별시 동작구 만양로 17, 1층 101, 102호 (상도동)",126.947208,37.506228,5,상도역 7호선,378,1970-01-01,영업/정상
0,"서울특별시 동작구 만양로14다길 17 (노량진동, 1층)",126.945571,37.511028,5,노량진역 1호선,454,1970-01-01,영업/정상
0,"서울특별시 동작구 사당로9나길 9, 1층 (사당동)",126.970106,37.486151,1,남성역 7호선,166,1970-01-01,영업/정상
0,"서울특별시 동작구 사당로17가길 13, 1층 (사당동)",126.973432,37.483803,3,남성역 7호선,248,1970-01-01,영업/정상
0,"서울특별시 동작구 만양로14가길 8, 1층 (노량진동)",126.945279,37.511925,4,노량진역 1호선,366,1970-01-01,영업/정상
0,"서울특별시 동작구 사당로28길 51, 1층 (사당동)",126.979503,37.480121,5,사당역 4호선,341,1970-01-01,영업/정상
0,"서울특별시 동작구 동작대로25길 16, 지하1, 1층, 2층 (사당동)",126.980975,37.486331,4,총신대입구역 4호선,121,1970-01-01,영업/정상
0,"서울특별시 동작구 상도로 264, 1층 (상도1동)",126.947127,37.503577,3,상도역 7호선,105,1970-01-01,영업/정상
0,"서울특별시 동작구 상도로37길 4, 1층 (상도1동)",126.948418,37.502980,3,상도역 7호선,45,1970-01-01,영업/정상
0,"서울특별시 동작구 사당로 196, 1층 101호 (사당동, 거목프리미엄상가)",126.969894,37.484930,1,남성역 7호선,91,1970-01-01,영업/정상


In [36]:
dj_sw8.to_csv('./data/dongjak_sw8.csv',encoding='utf-8')

In [20]:
pln = dj_sw8.place_name.unique()
pln   #22개

array(['노들역 9호선', '신대방삼거리역 7호선', '남성역 7호선', '숭실대입구역 7호선', '노량진역 9호선',
       '이수역 7호선', '보라매역 7호선', '흑석역 9호선', '상도역 7호선', '노량진역 1호선', '동작역 9호선',
       '대방역 1호선', '사당역 4호선', '장승배기역 7호선', '총신대입구역 4호선', '신림역 2호선',
       '신대방역 2호선', '낙성대역 2호선', '사당역 2호선', '구반포역 9호선', '신풍역 7호선',
       '구로디지털단지역 2호선', '샛강역 9호선'], dtype=object)

In [21]:
nodle = dj_sw8.loc[dj_sw8.place_name == pln[0]]
print(len(nodle))
nodle

14


,cf_address,X,Y,nb_num,place_name,distance,open_closed
0,"서울특별시 동작구 노량진로 206, 1층 (노량진동, 백명트렌디타워)",126.948626,37.512740,3,노들역 9호선,395,폐업
0,"서울특별시 동작구 노량진로28길 13, 지상1층 (본동)",126.955227,37.512257,2,노들역 9호선,193,폐업
0,"서울특별시 동작구 노량진로24길 36, 지하1층 (본동)",126.954040,37.511662,2,노들역 9호선,139,폐업
0,"서울특별시 동작구 노량진로26길 16-1, 1층 (본동)",126.954274,37.511562,2,노들역 9호선,161,폐업
0,"서울특별시 동작구 노량진로 206, 백명트렌디타워 7층 (노량진동)",126.948626,37.512740,3,노들역 9호선,395,영업/정상
0,"서울특별시 동작구 노량진로30길 12 (본동,2층)",126.955423,37.512580,2,노들역 9호선,205,영업/정상
0,"서울특별시 동작구 만양로8길 50 (노량진동,외 1필지 지하9호)",126.947804,37.509630,5,노들역 9호선,576,영업/정상
0,"서울특별시 동작구 노량진로 223 (본동, 1층)",126.950786,37.512969,3,노들역 9호선,206,영업/정상
0,서울특별시 동작구 노량진로 200 (노량진동),126.947910,37.512896,3,노들역 9호선,459,영업/정상
0,"서울특별시 동작구 노량진로24길 36, 1층 (본동)",126.954040,37.511662,2,노들역 9호선,139,영업/정상
